<a href="https://colab.research.google.com/github/karthikravi123/pytorch_practice/blob/main/Components_of_custom_data_loader_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1.**Dataset**(torch.utils.daata.Dataset) -dataset repsonsible for loading and transforming  
2.**Dataloader** (torch.utils.dat.DataLoader) -
wrappers the data set into an iterable objet that loads data in batches  
(handles batching and multiple proccess)  


Components_of_custom_data_loader_pytorch - advantanges
1.it stabilizes training means gradient updates happen more frequently.  
2. it reduces memory usuage,it doesnt load all the data at once  
3.it generalizes the model,by shiffling prevents overfitting ,with help of data loader  
This process is called mini batch  bcuz of instance of loading  all the data at once ,we are loading in small batches,mini batch introduces randomness  due to shuffle


1.Data gathering  
2.Data preprocessing  
3.Feature Engineering  
4.Model Training (create own model instead of scikit learn)  
5.Testing  


In [10]:
##dataset used
#https://www.kaggle.com/datasets/mirichoi0218/insurance

In [11]:
!pip install kaggle

In [12]:
import kagglehub
import os

path  = kagglehub.dataset_download("mirichoi0218/insurance")
path

Using Colab cache for faster access to the 'insurance' dataset.


'/kaggle/input/insurance'

In [13]:
os.listdir('/kaggle/input/insurance')


['insurance.csv']

In [14]:
import pandas as pd


In [15]:
df = pd.read_csv(os.path.join(path,'insurance.csv'))

In [16]:
print(type(df))

<class 'pandas.core.frame.DataFrame'>


In [17]:
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [18]:
df.describe()

,age,bmi,children,charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [20]:
import torch
import torch.nn as nn
##custom neural network
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.model_selection import train_test_split

In [21]:
#split dataset before encoding
train_df,test_df  = train_test_split(df,test_size=0.2,random_state=42)

In [22]:
##encode categorical variable
label_encoder = {}
for col in ['sex','smoker','region']:
  le = LabelEncoder()
  train_df[col] = le.fit_transform(train_df[col])
  test_df[col] = le.transform(test_df[col])
  label_encoder[col] = le



In [23]:
label_encoder

{'sex': LabelEncoder(), 'smoker': LabelEncoder(), 'region': LabelEncoder()}

In [24]:
##features and target

X_train = train_df.drop(columns=['charges'])
y_train = train_df['charges']

In [25]:

X_test = test_df.drop(columns=['charges'])
y_test = test_df['charges']

In [26]:
y_train.head()

,charges
560,9193.83850
1285,8534.67180
1142,27117.99378
969,8596.82780
486,12475.35130


In [27]:
#normalize features

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

pytorch model doesnt work with numpy pandas  with tensor
convert numpy pandas to tensor

In [28]:
##convert to tensors

X_train_tensor = torch.tensor(X_train,dtype = torch.float32)

y_train_tensor = torch.tensor(y_train.values,dtype = torch.float32).view(-1,1)
##change the view - flatten it

X_test_tensor = torch.tensor(X_test,dtype = torch.float32)
y_test_tensor = torch.tensor(y_test.values,dtype = torch.float32).view(-1,1)


.view(-1, 1)

What it does: Reshapes a PyTorch tensor.

-1 tells PyTorch: "figure out this dimension automatically based on the other dimension."

1 turns it into a column vector.

In [29]:
##Define neural network model

class SimpleNNRegresionModel(nn.Module):
  def __init__(self,input_dim):
    super(SimpleNNRegresionModel,self).__init__()
    self.network = nn.Sequential(
    nn.Linear(input_dim,64),
    nn.ReLU(),
    nn.Linear(64,128),
    nn.ReLU(),
    nn.Linear(128,1)
    )

  def forward(self,x):
    return self.network(x)


In [30]:
X_train_tensor.shape

torch.Size([1070, 6])

In [31]:
input_dim = X_train_tensor.shape[1]
model = SimpleNNRegresionModel(input_dim)

In [32]:
print(model)

SimpleNNRegresionModel(
  (network): Sequential(
    (0): Linear(in_features=6, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=1, bias=True)
  )
)


In [33]:
##loss and optimizer

#standard variable name used in loss function
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(),lr=0.01)


In [34]:
##training loop

epochs = 10000

for epoch in range(epochs):
  ##in model train - tells pytorch we eant to calculate gradients,store history  model training
  ##previous weights,what i s currrent weight ,pytorch tracks
  model.train()

  optimizer.zero_grad()
  prediction = model(X_train_tensor)
  loss = criterion(prediction,y_train_tensor)
  #calculate  gradients with loss back propogration
  loss.backward()

  #model.eval test model-pytorch dont want save or access it

  ##with optimizer update weights and gradients
  optimizer.step()

  if (epoch+10)%100 == 0 :
    print(f"epoch [{epoch+1}], loss:{loss.item():.4f}")

epoch [91], loss:51216672.0000
epoch [191], loss:32059408.0000
epoch [291], loss:29409640.0000
epoch [391], loss:27310218.0000
epoch [491], loss:26055912.0000
epoch [591], loss:25236546.0000
epoch [691], loss:24563186.0000
epoch [791], loss:23916280.0000
epoch [891], loss:23385154.0000
epoch [991], loss:22960606.0000
epoch [1091], loss:22394572.0000
epoch [1191], loss:21930372.0000
epoch [1291], loss:21557374.0000
epoch [1391], loss:21222316.0000
epoch [1491], loss:20918196.0000
epoch [1591], loss:20597158.0000
epoch [1691], loss:20251260.0000
epoch [1791], loss:19928100.0000
epoch [1891], loss:19654558.0000
epoch [1991], loss:19370800.0000
epoch [2091], loss:19081170.0000
epoch [2191], loss:18759218.0000
epoch [2291], loss:18465904.0000
epoch [2391], loss:18090140.0000
epoch [2491], loss:17667552.0000
epoch [2591], loss:17298196.0000
epoch [2691], loss:16980872.0000
epoch [2791], loss:16669924.0000
epoch [2891], loss:16389348.0000
epoch [2991], loss:16053434.0000
epoch [3091], loss:15

In [35]:
##model evaluation

model.eval()

##detach from memory and convert to numpy
y_pred = model(X_test_tensor).detach().numpy()

In [36]:
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
y_test_numpy = y_test_tensor.numpy()

##calcuate metrics
mse = mean_squared_error(y_test_numpy,y_pred)
rmse = mse ** 0.5
mae =mean_absolute_error(y_test_numpy,y_pred)
r2 = r2_score(y_test_numpy,y_pred)
print(f"mse {mse} \n , mae {mae},\n, r2 {r2}")


mse 36993496.0 
 , mae 4125.330078125,
, r2 0.7617146372795105


In [37]:
def predict_charges(age,sex,bmi,children,smoker,region):
  input_data = pd.DataFrame([[age,sex,bmi,children,smoker,region]],
               columns= ['age','sex','bmi','children','smoker','region'])
    ##encode categorical variable

  for col in ['sex','smoker','region']:
    input_data[col] = label_encoder[col].transform(input_data[col])
  input_data= scaler.transform(input_data)
  input_tensor = torch.tensor(input_data,dtype=torch.float32)
  predict_charge = model(input_tensor).item()
  return predict_charge

In [38]:
predicted = predict_charges(age=30, sex='male', bmi=25.0, children=2, smoker='yes', region='southwest')
print(f"predicted charges: ${predicted:.2f}")


predicted charges: $37194.25


Components of custom data loader


Creating custom Dataset in pytorch

init() - initialized the dataset,loads data,applied preprocessing  
len() - return the total numbers of samples in dataset  
getitem() - defines how to retrieve a single data sample when an index is provided

In [39]:
import torch
from torch.utils.data import Dataset,DataLoader

In [40]:
class InsuranceDataset(Dataset):
  def __init__(self,x,y):
    self.x = x
    self.y = y

  def __len__(self):
    return len(self.x)

  def __getitem__(self,idx):
    features = torch.tensor(self.x[idx],dtype=torch.float32)
    target = torch.tensor(self.y.values[idx],dtype=torch.float32)
    return features,target

In [41]:
dataset = InsuranceDataset(X_train,y_train)

In [50]:
##shuffle = data to be accessed in sequential order or random shuffle,parallel processing - num worker
dataloader = DataLoader(dataset,batch_size=32,shuffle=True,num_workers=4)

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [45]:
for batch_idx,(features,targets) in enumerate(dataloader):
  print(f"batch id {batch_idx+1}")
  print("features : ",features.shape)
  print("features : ",targets.shape)
  break

batch id 1
features :  torch.Size([32, 6])
features :  torch.Size([32])


1000 --total number of data

epoch -100  
bathc_size =1   
1000/1 = 1000  
100* 1000  

In [47]:
##shuffle = data to be accessed in sequential order or random shuffle
dataloader = DataLoader(dataset,batch_size=100,shuffle=True)

In [48]:
for batch_idx,(features,targets) in enumerate(dataloader):
  print(f"batch id {batch_idx+1}")
  print("features : ",features.shape)
  print("features : ",targets.shape)
  if batch_idx == 5:
    break

batch id 1
features :  torch.Size([100, 6])
features :  torch.Size([100])
batch id 2
features :  torch.Size([100, 6])
features :  torch.Size([100])
batch id 3
features :  torch.Size([100, 6])
features :  torch.Size([100])
batch id 4
features :  torch.Size([100, 6])
features :  torch.Size([100])
batch id 5
features :  torch.Size([100, 6])
features :  torch.Size([100])
batch id 6
features :  torch.Size([100, 6])
features :  torch.Size([100])


In [49]:
##training loop

epochs = 10000

for batch_idx , (batch_x,batch_y) in enumerate(dataloader):
  print(f"current batch : {batch_idx}")

  optimizer.zero_grad()
  prediction = model(batch_x)
  loss = criterion(prediction,batch_y)
  #calculate  gradients with loss back propogration
  loss.backward()
  print(f"batch [{batch_idx+1}/{epochs}],loss :{loss.item():.4f}")

  #model.eval test model-pytorch dont want save or access it

  ##with optimizer update weights and gradients
  optimizer.step()

  if (epoch+1)%100 == 0 :
    print(f"epoch [{epoch+1}], loss:{loss.item():.4f}")

current batch : 0
batch [1/10000],loss :218332288.0000
epoch [10000], loss:218332288.0000
current batch : 1
batch [2/10000],loss :211850352.0000
epoch [10000], loss:211850352.0000
current batch : 2
batch [3/10000],loss :196873376.0000
epoch [10000], loss:196873376.0000
current batch : 3
batch [4/10000],loss :233926192.0000
epoch [10000], loss:233926192.0000
current batch : 4
batch [5/10000],loss :149420032.0000
epoch [10000], loss:149420032.0000
current batch : 5
batch [6/10000],loss :212592656.0000
epoch [10000], loss:212592656.0000
current batch : 6
batch [7/10000],loss :200312624.0000
epoch [10000], loss:200312624.0000
current batch : 7
batch [8/10000],loss :215218048.0000
epoch [10000], loss:215218048.0000
current batch : 8
batch [9/10000],loss :210965568.0000
epoch [10000], loss:210965568.0000
current batch : 9
batch [10/10000],loss :178613840.0000
epoch [10000], loss:178613840.0000
current batch : 10
batch [11/10000],loss :142999376.0000
epoch [10000], loss:142999376.0000


/usr/local/lib/python3.12/dist-packages/torch/nn/modules/loss.py:634: UserWarning: Using a target size (torch.Size([100])) that is different to the input size (torch.Size([100, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.12/dist-packages/torch/nn/modules/loss.py:634: UserWarning: Using a target size (torch.Size([70])) that is different to the input size (torch.Size([70, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
